In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [6]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

In [7]:
# X_train=pd.read_csv('/kaggle/input/kernel-methods-ammi-2020/Xtr.csv', sep=',') #we use this dataset to train our model
# Y_train=pd.read_csv('/kaggle/input/kernel-methods-ammi-2020/Ytr.csv', sep=',') #we use this dataset to train our model
# X_test=pd.read_csv('/kaggle/input/kernel-methods-ammi-2020/Xte.csv', sep=',') #we will use this data set later to validate our model

In [8]:
X_train.tail()

Id                                                seq
1995  1995  TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996  1996  ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997  1997  TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998  1998  CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...
1999  1999  TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...

In [9]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)


In [10]:
# X_train['len'] = X_train.seq.apply(lambda x : len(x))

In [11]:
X_train.head()

Id                                                seq
0   0  GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1   1  CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2   2  GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3   3  GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4   4  GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...

In [12]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

In [13]:
#  X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])
for i in range(101):
    X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[i])
    
# X_train['seq_101'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[101])


In [14]:
X_train.head()

Id                                                seq  seq_0  seq_1  seq_2  \
0   0  GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...      7      1      7   
1   1  CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...      3      7      7   
2   2  GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...      7      1      3   
3   3  GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...      7      3      3   
4   4  GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...      7      3      1   

   seq_3  seq_4  seq_5  seq_6  seq_7  ...  seq_91  seq_92  seq_93  seq_94  \
0      7      7      7      3     20  ...       3      20       7       7   
1      3      3     20      7      7  ...       7      20       3       3   
2      1      1      3      7      3  ...       3       7       1       1   
3     20      3      3      3     20  ...       7       1       7      20   
4      3     20      1      3     20  ...       1       3      20       7   

   seq_95  seq_96  seq_97  seq_98  seq_99  seq_100  
0      20       7       7       3       1        7  
1       1       7       7      20       7        7  
2       7      20       7       3       7       20  
3       7       3      20       1       1        1  
4      20      20       1       3       3        3  

[5 rows x 103 columns]

In [15]:
X = X_train.drop(['seq', 'Id'], axis=1)
X_t = X_test.drop(['seq', 'Id'], axis=1)


In [16]:
y = Y_train.Bound

In [21]:
class LogisticRegressionBinary():
    def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.batch_size = batch_size
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid_func(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        y = self.trans_y(y)
        X = self.__add_intercept(X)
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid_func(z)
                        
            rand = np.random.choice(y.size, self.batch_size).squeeze()
            gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
            self.theta -= self.lr * gradient
            #print('theta and grad',self.theta.shape ,  gradient.shape )
            if(self.verbose == True and i % 100 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
    
    def predict_probability(self, X):
        X = self.__add_intercept(X)
    
        return self.__sigmoid_func(np.dot(X, self.theta))
    
    def predict(self, X, threshold=.5):
          return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
    def Accuracy_check(self,X,y):
        return np.mean(self.predict(X)==y)
    
    def trans_y(self, y):
        if isinstance(y, pd.Series):
            y = y.values
        if isinstance(y, list):
            y = np.array(y)
        return y

# Cross Validation

In [40]:
from sklearn.model_selection import KFold 

In [54]:
kfold=KFold(n_splits=5)
X_cross = X.values
y_cross = y.values

In [56]:
# import ipdb

In [61]:
accuracy = []
for i, (train_index, validate_index) in enumerate(kfold.split(X[:50])):
    
#     ipdb.set_trace()
    X_train, y_train = X_cross[train_index], y_cross[train_index]
    X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
    
    model = LogisticRegressionBinary()
    model.fit(X_train, y_train)
    
    accuracy.append(model.Accuracy_check(X_valid, y_valid))
    print(f'accurracy fold {i}: {accuracy[i]}')

print(f'Average accuracy is : {np.mean(accuracy)}')

accurracy fold 0: 0.6
accurracy fold 1: 0.4
accurracy fold 2: 0.7
accurracy fold 3: 0.2
accurracy fold 4: 0.5
Average accuracy is : 0.48


# Predictions

In [ ]:
y_pred = model.predict(X_t)

In [ ]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

In [ ]:
sample['Bound'] = y_pred

In [ ]:
sample.tail()

In [ ]:
sample.to_csv('./baseline.csv', index=False)